In [1]:
import os
import pyodbc
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};\
                       Server=lasr-sqldb-prd-im,17001;\
                       Database=IM_S_FAMIS_S;\
                       Trusted_Connection=yes;')

In [3]:
sql_query = """
select
RAPORTFOLIOID,
RTNPERIODENDDT,
RTNPERIODTYPCD,
RATETBLCD
from IM_I_IDWBRRADM_S.dbo.RA_IDX_RTN
where RATETBLCD = 'CG'
"""

# Save the data 
data = pd.read_sql(sql_query, cnxn)

# Close the connection with LASR
cnxn.close()

In [4]:
print(data.shape)
data.head()

(11367226, 4)


,RAPORTFOLIOID,RTNPERIODENDDT,RTNPERIODTYPCD,RATETBLCD
0,286398,2007-08-31,LIFE,CG
1,280112,2004-08-31,LIFE,CG
2,286467,1997-06-30,LIFEPRD,CG
3,288437,2014-06-30,1MO,CG
4,289901,2017-01-31,LIFE,CG


In [5]:
data['postdt_new'] = data['RTNPERIODENDDT'].apply(lambda dt: dt.replace(day=1))
data.head()

,RAPORTFOLIOID,RTNPERIODENDDT,RTNPERIODTYPCD,RATETBLCD,postdt_new
0,286398,2007-08-31,LIFE,CG,2007-08-01
1,280112,2004-08-31,LIFE,CG,2004-08-01
2,286467,1997-06-30,LIFEPRD,CG,1997-06-01
3,288437,2014-06-30,1MO,CG,2014-06-01
4,289901,2017-01-31,LIFE,CG,2017-01-01


In [6]:
#check no duplicate records
print(data.drop_duplicates(subset=['RAPORTFOLIOID', 'postdt_new', 'RTNPERIODTYPCD'], keep='first', inplace=False).count())

RAPORTFOLIOID     11367226
RTNPERIODENDDT    11367226
RTNPERIODTYPCD    11367226
RATETBLCD         11367226
postdt_new        11367226
dtype: int64


In [7]:
df_final = data.groupby(['postdt_new', 'RTNPERIODTYPCD'])['RAPORTFOLIOID'].count().reset_index()
print(df_final.shape)
df_final.head()

(22114, 3)


,postdt_new,RTNPERIODTYPCD,RAPORTFOLIOID
0,1912-12-01,LIFE,1
1,1912-12-01,LIFEPRD,1
2,1913-01-01,1MO,1
3,1913-01-01,LIFE,1
4,1913-01-01,LIFEPRD,1


In [8]:
df_final.rename(columns={'RAPORTFOLIOID':'RAPortfolioID_Cnt'}, inplace=True)
df_final.head()

,postdt_new,RTNPERIODTYPCD,RAPortfolioID_Cnt
0,1912-12-01,LIFE,1
1,1912-12-01,LIFEPRD,1
2,1913-01-01,1MO,1
3,1913-01-01,LIFE,1
4,1913-01-01,LIFEPRD,1


In [9]:
df_final.to_csv('summary_RA_IDX_RTN.csv')